# Log fast.ai metrics to neptune

## Prerequisites
Create your databunch.

In [ ]:
from fastai.vision import *
mnist = untar_data(URLs.MNIST_TINY)
tfms = get_transforms(do_flip=False)
data = (ImageItemList.from_folder(mnist)
    .split_by_folder()
    .label_from_folder()
    .transform(tfms, size=32)
    .databunch()
    .normalize(imagenet_stats))

## Create the `learner` find your optimal learning rate and plot it

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.lr_find()
learn.recorder.plot()

![image1](https://gist.githubusercontent.com/jakubczakon/fce515fb2ff90a7b61d81edb444898bc/raw/70b214b5ea2fdc42fc83c390d43f85a943d0e030/fastai_recorder.png)

## Create an experiment and add `neptune_monitor` callback

In [ ]:
import neptune
from neptunecontrib.monitoring.fastai import NeptuneMonitor

neptune.init(project_qualified_name='USER_NAME/PROJECT_NAME')

with neptune.create_experiment(params={'lr': 1e-2}):
    monitor = NeptuneMonitor()
    learn.callbacks.append(neptune_monitor)
    learn.fit_one_cycle(20, 1e-2)

## Monitor your fast.ai training in neptune
Now you can watch your fast.ai model training in neptune!

Go to the [experiment link](https://app.neptune.ml/neptune-ml/neptune-examples/e/NEP-493/charts) to see for yourself.

![image2](https://gist.githubusercontent.com/jakubczakon/f754769a39ea6b8fa9728ede49b9165c/raw/c74da877561b629773976ab07139e9fe3656d17b/contrib_fastai.png)

## Full fast.ai monitor script

In [ ]:
from fastai.vision import *
import neptune
from neptunecontrib.monitoring.fastai import NeptuneMonitor

neptune.init(project_qualified_name='USER_NAME/PROJECT_NAME')

mnist = untar_data(URLs.MNIST_TINY)
tfms = get_transforms(do_flip=False)

data = (ImageItemList.from_folder(mnist)
        .split_by_folder()
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch()
        .normalize(imagenet_stats))

learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.lr_find()
learn.recorder.plot()
    
with neptune.create_experiment(params={'lr': 1e-2}):
    monitor = NeptuneMonitor()
    learn.callbacks.append(neptune_monitor)
    learn.fit_one_cycle(20, 1e-2)